In [2]:
pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/9d/10/f01efcd8f34cce0adb59c9feebdd092bd7305d35e77400e9999557112ade/pymongo-4.6.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/87/a1/8c5287991ddb8d3e4662f71356d9656d91ab3a36618c3dd11b280df0d255/dnspython-2.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.8 kB ? eta -:--:--
   ----------------- ---------------------- 204.8/472.8 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 472.8/472.8 kB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   --------------------------------------- 307.7/307.7 kB 19.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [89]:
from pymongo import MongoClient
import pyarrow as pa
import pyarrow.parquet as pq
import pprint as pprint

In [54]:
mongo = MongoClient(port = 27017)
db = mongo.classDB

In [55]:
print(mongo.list_database_names())

['TaylorSwift_streamingdata', 'admin', 'config', 'local']


In [56]:
TS_db = mongo['TaylorSwift_streamingdata']

In [57]:
print(TS_db.list_collection_names())

['Taylor_swift_tracks', 'Taylor_Swift_Features', 'streamingdata']


In [58]:
TS_features = TS_db['Taylor_Swift_Features']
TS_tracks = TS_db['Taylor_swift_tracks']
TS_sd = TS_db['streamingdata']

In [59]:
print(TS_tracks.find_one())

{'_id': ObjectId('65f275f59ce574ddc5ac8c21'), '': 0, 'album_id': '1o59UpKw81iHR0HPiSkJR0', 'album_name': "1989 (Taylor's Version) [Deluxe]", 'id': '4WUepByoeqcedHoYhSNHRt', 'name': "Welcome To New York (Taylor's Version)", 'release_date': datetime.datetime(2023, 10, 27, 0, 0), 'duration': '00:03:32', 'track_number': 1, 'popularity': 71, 'explicit': False, 'uri': 'spotify:track:4WUepByoeqcedHoYhSNHRt'}


In [60]:
print(TS_sd.find_one())

{'_id': ObjectId('65f275a59ce574ddc5ac8935'), 'Song Title': 'Cruel Summer', 'Total Streams': 1828886612, 'Daily Streams': 4911187}


In [61]:
print(TS_features.find_one())

{'_id': ObjectId('65f275d39ce574ddc5ac8b2a'), '': 0, 'album_id': '1o59UpKw81iHR0HPiSkJR0', 'album_name': "1989 (Taylor's Version) [Deluxe]", 'id': '4WUepByoeqcedHoYhSNHRt', 'track_name': "Welcome To New York (Taylor's Version)", 'danceability': 0.757, 'energy': 0.61, 'key': 7, 'loudness': -4.84, 'mode': 1, 'speechiness': 0.0327, 'acousticness': 0.00942, 'instrumentalness': 3.66e-05, 'liveness': 0.367, 'valence': 0.685, 'tempo': 116.998}


In [27]:
#length of TS_tracks collection
print(TS_tracks.count_documents({}))

246


In [28]:
#length of TS_sd collection
print(TS_sd.count_documents({}))

500


In [62]:
#length of TS_features collection
print(TS_features.count_documents({}))

246


In [120]:
import pandas as pd
import numpy as np
array_streamingdata = []
array_num = np.arange(0,500,1, dtype = int)
for doc in TS_sd.find({}):
    #pprint.pprint(doc)
    array_streamingdata.append(doc)
TS_streamingdata_df = pd.DataFrame(data = array_streamingdata, index = np.arange(0, 500, 1, dtype = int) )
TS_streamingdata_df.convert_dtypes(infer_objects = False, convert_string = True)
New_TS_Streamingdata=TS_streamingdata_df[["Song Title", "Total Streams", "Daily Streams"]]
New_TS_Streamingdata = New_TS_Streamingdata.convert_dtypes(infer_objects = True, convert_string = True)
New_TS_Streamingdata.dtypes

Song Title       object
Total Streams     Int64
Daily Streams     Int64
dtype: object

In [136]:
tstracks = []
for coll in TS_tracks.find({}):
    tstracks.append(coll)
TS_trackinfo_df = pd.DataFrame(data=tstracks, index = np.arange(0, 246, 1, dtype=int))
TS_trackinfo_df.astype({"album_id": 'string', "album_name":'string'}).dtypes
new_TS_trackinfo = TS_trackinfo_df[["album_name", "name","release_date","duration","track_number","popularity","explicit","uri"]]
new_TS_trackinfo

,album_name,name,release_date,duration,track_number,popularity,explicit,uri
0,1989 (Taylor's Version) [Deluxe],Welcome To New York (Taylor's Version),2023-10-27,00:03:32,1,71,False,spotify:track:4WUepByoeqcedHoYhSNHRt
1,1989 (Taylor's Version) [Deluxe],Blank Space (Taylor's Version),2023-10-27,00:03:51,2,72,False,spotify:track:0108kcWLnn2HlH2kedi1gn
2,1989 (Taylor's Version) [Deluxe],Style (Taylor's Version),2023-10-27,00:03:51,3,73,False,spotify:track:3Vpk1hfMAQme8VJ0SNRSkd
3,1989 (Taylor's Version) [Deluxe],Out Of The Woods (Taylor's Version),2023-10-27,00:03:55,4,72,False,spotify:track:1OcSfkeCg9hRC2sFKB4IMJ
4,1989 (Taylor's Version) [Deluxe],All You Had To Do Was Stay (Taylor's Version),2023-10-27,00:03:13,5,70,False,spotify:track:2k0ZEeAqzvYMcx9Qt5aClQ
...,...,...,...,...,...,...,...,...
241,Taylor Swift,Our Song,2006-10-24,00:03:21,11,70,False,spotify:track:1j6gmK6u4WNI33lMZ8dC1s
242,Taylor Swift,I'm Only Me When I'm With You,2006-10-24,00:03:33,12,61,False,spotify:track:7CzxXgQXurKZCyHz9ufbo1
243,Taylor Swift,Invisible,2006-10-24,00:03:23,13,59,False,spotify:track:1k3PzDNjg38cWqOvL4M9vq
244,Taylor Swift,A Perfectly Good Heart,2006-10-24,00:03:40,14,58,False,spotify:track:0YgHuReCSPwTXYny7isLja


In [138]:
tsfeatures = []
for docu in TS_features.find({}):
    tsfeatures.append(docu)
TS_features_df = pd.DataFrame(data = tsfeatures, index = np.arange(0, 246,1, dtype =int))
new_ts_features = TS_features_df[["album_name","track_name","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]]
new_ts_features

,album_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1989 (Taylor's Version) [Deluxe],Welcome To New York (Taylor's Version),0.757,0.610,7,-4.840,1,0.0327,0.009420,0.000037,0.3670,0.685,116.998
1,1989 (Taylor's Version) [Deluxe],Blank Space (Taylor's Version),0.733,0.733,0,-5.376,1,0.0670,0.088500,0.000000,0.1680,0.701,96.057
2,1989 (Taylor's Version) [Deluxe],Style (Taylor's Version),0.511,0.822,11,-4.785,0,0.0397,0.000421,0.019700,0.0899,0.305,94.868
3,1989 (Taylor's Version) [Deluxe],Out Of The Woods (Taylor's Version),0.545,0.885,0,-5.968,1,0.0447,0.000537,0.000056,0.3850,0.206,92.021
4,1989 (Taylor's Version) [Deluxe],All You Had To Do Was Stay (Taylor's Version),0.588,0.721,0,-5.579,1,0.0317,0.000656,0.000000,0.1310,0.520,96.997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Taylor Swift,Our Song,0.668,0.672,2,-4.931,1,0.0303,0.111000,0.000000,0.3290,0.539,89.011
242,Taylor Swift,I'm Only Me When I'm With You,0.563,0.934,8,-3.629,1,0.0646,0.004520,0.000807,0.1030,0.518,143.964
243,Taylor Swift,Invisible,0.612,0.394,7,-5.723,1,0.0243,0.637000,0.000000,0.1470,0.233,96.001
244,Taylor Swift,A Perfectly Good Heart,0.483,0.751,4,-5.726,1,0.0365,0.003490,0.000000,0.1280,0.268,156.092
